# Manual Testing

See `readme.md` for setup instructions.

Then make sure this jupyter notebook is restarted before rebuilding, or `maturin` will fail to copy the fresh `.pyd` library into `.env/Lib/megalinref/` .

In [1]:
import json
import megalinref as mlr
from pathlib import Path
import pandas as pd
import numpy as np
import util.unit_conversion as uc
import math

In [2]:
path_to_temp_data = Path(".").resolve().joinpath("../tests/temporary_test_data")
path_to_temp_data.mkdir(exist_ok=True)
path_to_gitignore = path_to_temp_data / ".gitignore"
path_to_json_data = path_to_temp_data / "road_network.json"
path_to_binary_data = path_to_temp_data / "road_network.bin"

## Make `.gitignore`

In [3]:
path_to_gitignore.write_text("*\n!.gitignore")

13

## JSON Cache

In [4]:
if not path_to_json_data.exists():
    print(f"There is no data cached at {path_to_json_data}")
    # Fetch new data
    cached_json_data = mlr.download_fresh_data_as_json()
    # Cache the data
    with path_to_json_data.open("w") as f:
        json.dump(cached_json_data, f)
else:
    # Use cached data
    with path_to_json_data.open("r") as f:
        cached_json_data = json.load(f)

There is no data cached at E:\GitProjects\megalinref\tests_notebooks\..\tests\temporary_test_data\road_network.json
........................................................................................................................................................................................
........................................................................................................................................................................................
Download Completed. received 184060 records


In [5]:
cached_json_data["features"][1]

{'type': 'Feature',
 'geometry': {'type': 'LineString',
  'coordinates': [[116.01613363394955, -31.90864177309936],
   [116.01564263139306, -31.90885735305153]]},
 'properties': {'ROAD': 'X001',
  'START_SLK': 0.03,
  'END_SLK': 0.08,
  'CWY': 'Single',
  'NETWORK_TYPE': 'Crossover',
  'START_TRUE_DIST': 0.03,
  'END_TRUE_DIST': 0.08}}

In [6]:
df = pd.DataFrame.from_records([item["properties"] for item in cached_json_data["features"]])

In [7]:
df.duplicated().sum()

0

In [8]:
cways = set()
network_types = set()
for item in cached_json_data["features"]:
    cways.add(item["properties"]["CWY"])
    network_types.add(item["properties"]["NETWORK_TYPE"])
print(cways)
print(network_types)

{'Right', 'Left', 'Single'}
{'Proposed Road', 'Miscellaneous Road', 'Main Roads Controlled Path', 'Local Road', 'Crossover', 'State Road'}


## Binary Cache

In [9]:
if not path_to_binary_data.exists():
    print(f"There is no binary data cached at {path_to_binary_data}")
    # Fetch new data
    cached_binary_data = mlr.Lookup.from_dict(cached_json_data).to_binary()
    # Cache the data
    with path_to_binary_data.open("wb") as f:
        f.write(cached_binary_data)
else:
    # Use cached data
    with path_to_binary_data.open("rb") as f:
        cached_binary_data = f.read()

There is no binary data cached at E:\GitProjects\megalinref\tests_notebooks\..\tests\temporary_test_data\road_network.bin


In [11]:
lookup = mlr.Lookup.from_binary(cached_binary_data)
lookup

In [12]:
result = lookup.road_slk_from_coordinate(
    lat = -31.89006203575722,
    lon = 115.80183730752809,
    carriageways  = mlr.Cwy["All"],
    network_types = mlr.NetworkType["All"],
    roads = []
)
result

{'feature': {'ROAD': 'H016',
  'CWY': 'Left',
  'START_SLK': 9.84,
  'END_SLK': 10.68,
  'START_TRUE_DIST': 9.84,
  'END_TRUE_DIST': 10.68,
  'NETWORK_TYPE': 'State Road'},
 'slk': 9.99999981522603,
 'true': 9.99999981522603,
 'distance_metres': 1.1852499613658425e-09}